In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import midii

In [3]:
# --- Constants and Helper Functions ---
# Use constants consistent with your algorithm/data
TPQN = 480
# Define QuantaList in beats (matching pseudocode)
QuantaListBeats = [4, 2, 1, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625]
TargetUnitBeat = 0.125 # Example: 1/32 note

In [ ]:
def tick_to_beats(ticks, tpqn):
    return ticks / float(tpqn)

def beat_to_ticks(beats, tpqn):
    return int(round(beats * tpqn)) # Return integer ticks

# --- Placeholder Data ---
def generate_original_deltas(num_events=500):
    """Generates a list of somewhat irregular delta times."""
    deltas = []
    # Simulate some notes around 16th/32nd notes with variation
    base_delta = TPQN / 8 # 32nd note ticks
    for _ in range(num_events):
        # Add variation, ensure non-negative
        delta = max(0, base_delta + random.randint(-base_delta//2, base_delta//2))
        # Occasionally add a longer delta
        if random.random() < 0.05:
            delta *= random.randint(2, 6)
        # Add some zero deltas
        if random.random() < 0.1:
            delta = 0
        deltas.append(int(delta))
    return deltas

In [ ]:
def calculate_absolute_times(delta_times):
    """Calculates absolute times from delta times."""
    return np.cumsum(np.array(delta_times, dtype=np.int64))

# --- Simulation of Quantization Logic ---
# (Based on pseudocode - replace with your actual midii calls)

def internal_quantize_sim(input_delta_ticks, target_unit_beat):
    """Simulates the InternalQuantize function from pseudocode."""
    if input_delta_ticks < 0: input_delta_ticks = 0 # Safety clamp

    quantized_beats_total = 0
    current_remainder_beats = tick_to_beats(input_delta_ticks, TPQN)
    error_beats = 0

    # Part 1: Greedy consumption
    index = 0
    while index < len(QuantaListBeats):
        current_quantum_beats = QuantaListBeats[index]
        while current_remainder_beats >= current_quantum_beats:
            current_remainder_beats -= current_quantum_beats
            quantized_beats_total += current_quantum_beats

        if current_quantum_beats == target_unit_beat:
            break
        index += 1

    # Part 2: Round remainder
    quantized_remainder_beats = 0
    if current_remainder_beats < (target_unit_beat / 2.0):
        error_beats = current_remainder_beats - 0
        quantized_remainder_beats = 0
    else:
        error_beats = current_remainder_beats - target_unit_beat
        quantized_remainder_beats = target_unit_beat

    quantized_beats_total += quantized_remainder_beats
    quantized_delta_ticks = beat_to_ticks(quantized_beats_total, TPQN)
    error_ticks_for_next_step = beat_to_ticks(error_beats, TPQN)

    return quantized_delta_ticks, error_ticks_for_next_step

def quantize_naive(original_delta_list, target_unit_beat):
    """Simulates quantization WITHOUT error forwarding."""
    new_quantized_delta_list = []
    for original_delta in original_delta_list:
        if original_delta == 0:
            new_quantized_delta_list.append(0)
            continue
        # Quantize directly, ignore returned error
        quantized_delta, _ = internal_quantize_sim(original_delta, target_unit_beat)
        new_quantized_delta_list.append(quantized_delta)
    return new_quantized_delta_list

def quantize_with_EF(original_delta_list, target_unit_beat):
    """Simulates quantization WITH error forwarding (your algorithm)."""
    error_to_propagate_ticks = 0
    new_quantized_delta_list = []
    for original_delta in original_delta_list:
        if original_delta == 0:
            new_quantized_delta_list.append(0)
            # Decide if error resets on zero delta, e.g., error_to_propagate_ticks = 0
            continue

        time_adjusted_for_error = original_delta
        if error_to_propagate_ticks != 0:
            potential_time = time_adjusted_for_error + error_to_propagate_ticks
            time_adjusted_for_error = max(0, potential_time) # Clamp >= 0

        quantized_delta, current_step_error = internal_quantize_sim(
            time_adjusted_for_error, target_unit_beat
        )
        new_quantized_delta_list.append(quantized_delta)
        error_to_propagate_ticks = current_step_error
    return new_quantized_delta_list

# --- Generate Data for Plotting ---
original_deltas = generate_original_deltas(num_events=500)

# Calculate original absolute times
original_abs_times = calculate_absolute_times(original_deltas)

# Quantize using both methods
quantized_deltas_naive = quantize_naive(original_deltas, TargetUnitBeat)
quantized_deltas_with_EF = quantize_with_EF(original_deltas, TargetUnitBeat)

# Calculate absolute times for quantized versions
quantized_abs_times_naive = calculate_absolute_times(quantized_deltas_naive)
quantized_abs_times_with_EF = calculate_absolute_times(quantized_deltas_with_EF)

# Calculate drift (deviation from original absolute time)
drift_naive = quantized_abs_times_naive - original_abs_times
drift_with_EF = quantized_abs_times_with_EF - original_abs_times

# Use event index as x-axis for simplicity
event_index = np.arange(len(original_deltas))

# --- Plotting ---
fig, ax = plt.subplots(figsize=(12, 5))
fig.suptitle('Figure 3: Absolute Timing Drift Comparison', fontsize=14)

ax.plot(event_index, drift_naive, label='Quantized w/o Error Forwarding (Naive)', alpha=0.8, linewidth=1.5)
ax.plot(event_index, drift_with_EF, label='Quantized w/ Error Forwarding (Proposed)', alpha=0.8, linewidth=1.5)

ax.axhline(0, color='black', linestyle='--', linewidth=0.8, label='Original Timing')
ax.set_xlabel("Event Index")
ax.set_ylabel("Timing Drift (Quantized Abs Time - Original Abs Time) [ticks]")
ax.set_title(f"Drift Comparison (Quantization Unit = 1/{int(1/TargetUnitBeat)} Note, TPQN={TPQN})")
ax.grid(True, linestyle=':')
ax.legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show() # Use plt.savefig('figure3_drift_comparison.png', dpi=300) for paper
